In [1]:
from datasets import load_dataset
ds = load_dataset("facehuggerapoorv/resume-jd-match")

README.md:   0%|          | 0.00/411 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/29.0M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/8.07M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/6241 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [2]:
train_data = ds["train"]
test_data = ds["test"]

In [3]:
from sentence_transformers import SentenceTransformer, InputExample, losses

2025-09-26 11:34:50.678956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758886490.869172      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758886490.928205      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("/kaggle/input/model1/5_epochs",device=device)

In [11]:
import re

def split_jd_cv(text):
    # Tách bằng "the resume:"
    if "the resume:" in text:
        jd, cv = text.split("the resume:", 1)  # chỉ split lần đầu
        jd = jd.strip()
        cv = cv.strip()
        return jd, cv
    else:
        return text, None   # nếu ko có marker thì giữ nguyên

In [12]:
import re
import unicodedata

def preprocess_text(text):
    # 1. Unicode normalize (chuẩn hóa dấu, ký tự)
    text = unicodedata.normalize("NFC", text)
    
    # 2. Loại bỏ HTML tags
    text = re.sub(r"<[^>]*>", " ", text)
    
    # 3. Loại bỏ các ký hiệu/metadata không cần thiết (@@, ###, etc.)
    text = re.sub(r"[@#]{2,}", " ", text)
    
    # 4. Loại bỏ ký tự không phải chữ/số/dấu câu cơ bản
    text = re.sub(r"[^0-9a-zA-ZÀ-ỹ.,!?;:()\-\s]", " ", text)
    
    # 5. Lowercase nếu cần
    text = text.lower()
    
    # 6. Loại bỏ khoảng trắng thừa
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

In [13]:
def make_dataset(data):
    dataset = []
    for i in range(len(data["text"])):
        if(i%100==0):
            print(i)
        jd, cv = split_jd_cv(data['text'][i][:-22])
        if jd == None or cv == None:
            continue
        #jd = preprocess_text(jd)
        #cv = preprocess_text(cv)
        dataset.append(InputExample(texts=[jd,cv],label=float(data['idx_label'][i])))
    return dataset

In [16]:
import random
random.seed(42)
def label_to_score(label: str) -> float:
    if label == "No Fit":
        random.seed(42)
        return random.uniform(0.0, 0.5)
    elif label == "Potential Fit":
        random.seed(42)
        return random.uniform(0.5, 0.75)
    elif label == "Good Fit":
        random.seed(42)
        return random.uniform(0.75, 1.0)
    else:
        return 0.3

In [17]:
from torch.utils.data import DataLoader

# dataset của hugging face ko gán trực tiếp như pandas
train_data = train_data.map(lambda x: {"idx_label": label_to_score(x["label"])})
test_data = test_data.map(lambda x: {"idx_label": label_to_score(x["label"])})

Map:   0%|          | 0/6241 [00:00<?, ? examples/s]

Map:   0%|          | 0/1759 [00:00<?, ? examples/s]

In [18]:
train_data['idx_label'][0]

0.3197133992289419

In [19]:
from sentence_transformers import SentencesDataset

In [20]:
train_dataset = SentencesDataset(make_dataset(train_data),model)
#test_dataset = SentencesDataset(make_dataset(test_data))
print(train_dataset[0].label)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
0.3197133992289419


In [13]:
print(train_dataset[0].texts)

['for the given job description', '.']


In [21]:
train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=8,
    collate_fn=model.smart_batching_collate,
    pin_memory=True
)

In [22]:
print(model.device)

cuda:0


In [23]:
from torch.cuda.amp import autocast, GradScaler
from torch import amp
train_loss = losses.CosineSimilarityLoss(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


scaler = amp.GradScaler("cuda")

# 6. Training Loop
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

for epoch in range(5):
    model.train()
    for features, labels in train_dataloader:
        # features: list[dict], labels: tensor
        #features = features.to("cuda", non_blocking=True)  # nhanh hơn
        labels = labels.to("cuda", non_blocking=True)
        features = [{k: v.to(device) for k, v in f.items()} for f in features]

        optimizer.zero_grad()
        with amp.autocast("cuda"):
            loss = train_loss(features, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    print(f"Epoch {epoch+1} finished, loss={loss.item():.4f}")


Epoch 1 finished, loss=0.0010
Epoch 2 finished, loss=0.0007
Epoch 3 finished, loss=0.0065
Epoch 4 finished, loss=0.0000
Epoch 5 finished, loss=0.0016


In [24]:
import os
output_path = "./e5_finetuned_cvjd"

# Tạo thư mục nếu chưa tồn tại
os.makedirs(output_path, exist_ok=True)

# Save model
model.save(output_path)

In [25]:
import shutil

# Tạo file zip tên all_output.zip từ thư mục output/
shutil.make_archive('all_output', 'zip', '/kaggle/working/e5_finetuned_cvjd')

print("File all_output.zip đã tạo, bạn có thể tải xuống từ tab Files")

File all_output.zip đã tạo, bạn có thể tải xuống từ tab Files


In [26]:
test_cv = []
test_cp = []
true_label = []
for i in range(len(test_data)):
    jd, cv = split_jd_cv(test_data['text'][i][:-22])
    if jd == None or cv == None:
        continue
    jd = preprocess_text(jd)
    cv = preprocess_text(cv)
    test_cv.append(cv)
    test_cp.append(jd)
    true_label.append(test_data['label'][i])

In [ ]:
from sentence_transformers import util
correct = 0
for i in range(len(test_cp)):
    emb1 = model.encode(test_cp[i], convert_to_tensor=True)
    emb2 = model.encode(test_cv[i], convert_to_tensor=True)
    similarity = util.cos_sim(emb1, emb2)
    if similarity.item() < 0.5:
        if true_label[i] == 'No Fit':
            correct += 1
    elif similarity.item() < 0.75:
        if true_label[i] == 'Potential Fit':
            correct += 1
    else:
        if true_label[i] == 'Good Fit':
            correct += 1
print(correct/len(test_cp))

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]